In [58]:
# importamos las librerías que necesitamos
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

In [59]:
# Leer CSV 2
df2 = pd.read_csv("Files/Customer Loyalty History.csv")

In [60]:
columnas_traducidas2 = {
    'Loyalty Number': 'numero_cliente',
    'Country': 'pais',
    'Province': 'provincia',
    'City': 'ciudad',
    'Postal Code': 'codigo_postal',
    'Gender': 'genero',
    'Education': 'educacion',
    'Salary': 'salario',
    'Marital Status': 'estado_civil',
    'Loyalty Card': 'tarjeta_lealtad',
    'CLV': 'valoracion_cliente',
    'Enrollment Type': 'tipo_inscripcion',
    'Enrollment Year': 'ano_inscripcion',
    'Enrollment Month': 'mes_inscripcion',
    'Cancellation Year': 'ano_cancelacion',
    'Cancellation Month': 'mes_cancelacion'
}
df2.rename(columns= columnas_traducidas2, inplace=True)

In [61]:
df2.head()

,numero_cliente,pais,provincia,ciudad,codigo_postal,genero,educacion,salario,estado_civil,tarjeta_lealtad,valoracion_cliente,tipo_inscripcion,ano_inscripcion,mes_inscripcion,ano_cancelacion,mes_cancelacion
0,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
1,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,NaN,Divorced,Star,3839.61,Standard,2016,3,NaN,NaN
2,429460,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Single,Star,3839.75,Standard,2014,7,2018.0,1.0
3,608370,Canada,Ontario,Toronto,P1W 1K4,Male,College,NaN,Single,Star,3839.75,Standard,2013,2,NaN,NaN
4,530508,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,103495.0,Married,Star,3842.79,Standard,2014,10,NaN,NaN


Vamos a empezar gestionando los nulos viendo el % que tenemos con respecto al resto de datos

In [62]:
df2.isna().sum()/df2.shape[0]*100

numero_cliente         0.000000
pais                   0.000000
provincia              0.000000
ciudad                 0.000000
codigo_postal          0.000000
genero                 0.000000
educacion              0.000000
salario               25.321145
estado_civil           0.000000
tarjeta_lealtad        0.000000
valoracion_cliente     0.000000
tipo_inscripcion       0.000000
ano_inscripcion        0.000000
mes_inscripcion        0.000000
ano_cancelacion       87.650117
mes_cancelacion       87.650117
dtype: float64

Empezamos por el año y mes de cancelación

Como consideramos los Nan de la columna de año y mes como que no se han dado de baja, vamos a generar una columna que diga si el cliente esta activo o no y luego sustituirlos por 9999 en el caso del año y 99 en el caso de los meses

In [63]:
df2["estado_cliente"] = df2["ano_cancelacion"].apply(lambda x: "Activo" if pd.isna(x) else 'Cancelado')

In [64]:
print(f"El numero de nulos en año de cancelacion antes de convertirlos es {df2['ano_cancelacion'].isna().sum()}")
df2["ano_cancelacion"] = df2["ano_cancelacion"].fillna(9999)
print(f"El numero de nulos en año de cancelacion después de convertirlos es {df2['ano_cancelacion'].isna().sum()}")

El numero de nulos en año de cancelacion antes de convertirlos es 14670
El numero de nulos en año de cancelacion después de convertirlos es 0


In [65]:
print(f"El numero de nulos mes de cancelacion antes de convertirlos es {df2['mes_cancelacion'].isna().sum()}")
df2["mes_cancelacion"] = df2["mes_cancelacion"].fillna(99)
print(f"El numero de nulos mes de cancelacion después de convertirlos es {df2['mes_cancelacion'].isna().sum()}")

El numero de nulos mes de cancelacion antes de convertirlos es 14670
El numero de nulos mes de cancelacion después de convertirlos es 0


Comprobamos y ahora no tenemos nulos en esas dos columnas y vamos investigar la del salario pero primero veremos esos valores negativos

In [66]:
df2[df2["salario"]<0].shape

(20, 17)

In [81]:
df2[df2["salario"]<0].head(10)

,numero_cliente,pais,provincia,ciudad,codigo_postal,genero,educacion,salario,estado_civil,tarjeta_lealtad,valoracion_cliente,tipo_inscripcion,ano_inscripcion,mes_inscripcion,ano_cancelacion,mes_cancelacion,estado_cliente
1082,542976,Canada,Quebec,Montreal,H2Y 4R4,Male,High School or Below,-49830.0,Divorced,Star,24127.50,2018 Promotion,2018,3,9999.0,99.0,Activo
1894,959977,Canada,British Columbia,Vancouver,V5R 1W3,Female,Bachelor,-12497.0,Married,Aurora,9453.00,2018 Promotion,2018,3,9999.0,99.0,Activo
2471,232755,Canada,British Columbia,Vancouver,V1E 4R6,Female,Bachelor,-46683.0,Single,Nova,4787.81,2018 Promotion,2018,3,9999.0,99.0,Activo
3575,525245,Canada,British Columbia,Victoria,V10 6T5,Male,Bachelor,-45962.0,Married,Star,2402.33,2018 Promotion,2018,3,9999.0,99.0,Activo
3932,603070,Canada,British Columbia,West Vancouver,V6V 8Z3,Female,Bachelor,-19325.0,Single,Star,2893.74,2018 Promotion,2018,3,9999.0,99.0,Activo
4712,491242,Canada,British Columbia,Dawson Creek,U5I 4F1,Male,Bachelor,-43234.0,Married,Star,7597.91,2018 Promotion,2018,3,9999.0,99.0,Activo
6560,115505,Canada,Newfoundland,St. John's,A1C 6H9,Male,Bachelor,-10605.0,Married,Nova,5860.17,2018 Promotion,2018,4,9999.0,99.0,Activo
6570,430398,Canada,Newfoundland,St. John's,A1C 6H9,Male,Bachelor,-17534.0,Married,Nova,49423.80,2018 Promotion,2018,3,9999.0,99.0,Activo
7373,152016,Canada,Ontario,Toronto,P1J 8T7,Female,Bachelor,-58486.0,Married,Aurora,5067.21,2018 Promotion,2018,2,9999.0,99.0,Activo
8576,194065,Canada,Ontario,Sudbury,M5V 1G5,Female,Bachelor,-31911.0,Married,Nova,2888.85,2018 Promotion,2018,2,9999.0,99.0,Activo


Tenemos 20 clientes con valores negativos. Como hay valores negativos sin importar la educacion. Vamos a comparar el salario dependiendo de la educacion para ver si podemos sacar la conclusiÓn de que es un error y asi cambiarlos a posiivos

In [68]:
df2.groupby("educacion")["salario"].describe()

,count,mean,std,min,25%,50%,75%,max
educacion,,,,,,,,
Bachelor,10475.0,72451.061575,17207.352107,-58486.0,58711.00,71960.0,85808.0,105563.0
College,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Doctor,734.0,178608.897820,70466.188750,48109.0,124768.00,182143.5,236612.0,407228.0
High School or Below,782.0,61071.718670,12733.501155,-49830.0,52151.25,61915.0,71384.5,93875.0
Master,508.0,103757.848425,17527.938162,56414.0,88120.00,105487.0,118530.0,133080.0


Vemos que la educacion College tiene todo Nan, esto nos hace pensar que todos los que tienen nan en el salario corresponden a ese grupo de educacion. 

In [73]:
grupo_college = df2[df2["educacion"] == "College"]

In [75]:
grupo_college.isna().sum()/df2.shape[0]*100

numero_cliente         0.000000
pais                   0.000000
provincia              0.000000
ciudad                 0.000000
codigo_postal          0.000000
genero                 0.000000
educacion              0.000000
salario               25.321145
estado_civil           0.000000
tarjeta_lealtad        0.000000
valoracion_cliente     0.000000
tipo_inscripcion       0.000000
ano_inscripcion        0.000000
mes_inscripcion        0.000000
ano_cancelacion        0.000000
mes_cancelacion        0.000000
estado_cliente         0.000000
dtype: float64

Vamos a comprobar que el resto de grupos no tienen Nan

In [78]:
resto_grupos = df2[df2["educacion"] != "College"]

In [79]:
resto_grupos.isna().sum()

numero_cliente        0
pais                  0
provincia             0
ciudad                0
codigo_postal         0
genero                0
educacion             0
salario               0
estado_civil          0
tarjeta_lealtad       0
valoracion_cliente    0
tipo_inscripcion      0
ano_inscripcion       0
mes_inscripcion       0
ano_cancelacion       0
mes_cancelacion       0
estado_cliente        0
dtype: int64

Vamos a continuar con la investigación de los valores negativos

In [87]:
salarios_negativos = df2[df2["salario"]<=0]

In [88]:
salarios_positivos = df2[df2["salario"]>0]

In [89]:
salarios_negativos.groupby("educacion")["salario"].describe()

,count,mean,std,min,25%,50%,75%,max
educacion,,,,,,,,
Bachelor,19.0,-34786.052632,16080.943973,-58486.0,-46576.5,-39503.0,-19328.5,-9081.0
High School or Below,1.0,-49830.000000,NaN,-49830.0,-49830.0,-49830.0,-49830.0,-49830.0


In [90]:
salarios_positivos.groupby("educacion")["salario"].describe()

,count,mean,std,min,25%,50%,75%,max
educacion,,,,,,,,
Bachelor,10456.0,72645.926262,16590.634159,15609.0,58715.5,72026.0,85848.0,105563.0
Doctor,734.0,178608.897820,70466.188750,48109.0,124768.0,182143.5,236612.0,407228.0
High School or Below,781.0,61213.718310,12106.260467,21853.0,52209.0,61928.0,71398.0,93875.0
Master,508.0,103757.848425,17527.938162,56414.0,88120.0,105487.0,118530.0,133080.0


Como los datos estan comprendidos entre los rangos salariales de sus categorias vamos a pasarlos a positivo

In [91]:
def convertir_positivo(valor):
    if pd.isna(valor):
        return valor 
    if valor < 0:
        return abs(valor)
    else:
        return valor
    

In [92]:
df2["salario"] = df2["salario"].apply(convertir_positivo)

In [94]:
df2[df2["salario"]<=0].shape

(0, 17)

In [95]:
df2.groupby("educacion")["salario"].describe()

,count,mean,std,min,25%,50%,75%,max
educacion,,,,,,,,
Bachelor,10475.0,72577.254415,16667.023853,9081.0,58711.00,71960.0,85808.0,105563.0
College,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Doctor,734.0,178608.897820,70466.188750,48109.0,124768.00,182143.5,236612.0,407228.0
High School or Below,782.0,61199.161125,12105.354128,21853.0,52151.25,61915.0,71384.5,93875.0
Master,508.0,103757.848425,17527.938162,56414.0,88120.00,105487.0,118530.0,133080.0
